In [1]:
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from tqdm import tqdm
from sklearn.manifold import TSNE
import pickle as pk

In [2]:
# data read
author_emd = {}
dataset_emd = {}
entity_emd = {}
fAuthor = open("../data/0404author_node_embedding.txt", "r")

print('start')
for line in tqdm(fAuthor):
    line = line.strip().split(' ')
    temp_ebd = list(map(float,line[1:]))
    author_emd[line[0]] = np.array(temp_ebd)
fAuthor.close()

fDataset = open("../data/0404dataset_node_embedding.txt", "r")
for line in tqdm(fDataset):
    line = line.strip().split(' ')
    temp_ebd = list(map(float,line[1:]))
    dataset_emd[line[0]] = np.array(temp_ebd)
fDataset.close()


start


32383it [00:01, 30208.18it/s]
227it [00:00, 28382.13it/s]


In [3]:
# combine them
dataset_author_keys = list(dataset_emd.keys()) + list(author_emd.keys())
dataset_author_list = list(dataset_emd.values()) + list(author_emd.values())
dataset_author_array = np.array(dataset_author_list)

In [116]:
# kmeans
dataset_author_Kmeans = KMeans(
    n_clusters=5, random_state=0).fit(dataset_author_array)
print('kmeans over')

clusters = dataset_author_Kmeans.predict(dataset_author_array)

dataset_author_kmeans_label = {}
for i in range(len(dataset_author_keys)):
    dataset_author_kmeans_label[dataset_author_keys[i]] = clusters[i]

dataset_cluster = {}
author_cluster = {}
for i in dataset_emd.keys():
    dataset_cluster[int(i)] = dataset_author_kmeans_label[i]
for i in author_emd.keys():
    author_cluster[int(i)] = dataset_author_kmeans_label[i]


kmeans over


In [49]:
# tsne
dataset_author_embedded = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=100).fit_transform(dataset_author_array)

In [50]:
dataset_author_ebd = {}
for i in range(len(dataset_author_keys)):
    dataset_author_ebd[dataset_author_keys[i]] = dataset_author_embedded[i]
dataset_author_ebd

{'1385': array([-14.076514,   2.58017 ], dtype=float32),
 '2583': array([-14.074586 ,   2.5515714], dtype=float32),
 '856': array([-0.92049533, 35.87663   ], dtype=float32),
 '1409': array([51.2622 , 45.99125], dtype=float32),
 '1780': array([13.441985, 37.63287 ], dtype=float32),
 '807': array([-13.464289, -18.85491 ], dtype=float32),
 '2489': array([-37.72101,  59.58457], dtype=float32),
 '2245': array([-76.96482 , -53.586555], dtype=float32),
 '7': array([-76.96529 , -53.587563], dtype=float32),
 '565': array([73.71283, 44.93347], dtype=float32),
 '104': array([-83.33572 ,  23.900837], dtype=float32),
 '1216': array([-11.7518015,  -1.8481039], dtype=float32),
 '2475': array([ -9.417603, -27.369715], dtype=float32),
 '27': array([29.41565 , 75.691696], dtype=float32),
 '2649': array([42.476944, 85.29101 ], dtype=float32),
 '172': array([40.096645, 87.267784], dtype=float32),
 '2371': array([-69.90395 , -30.135979], dtype=float32),
 '795': array([84.03055 , 22.374523], dtype=float32),

In [51]:
dataset_position = {}
author_position = {}
for i in dataset_emd.keys():
    dataset_position[i] = dataset_author_ebd[i]
for i in author_emd.keys():
    author_position[i] = dataset_author_ebd[i]

pk.dump(dataset_position, open('dataset_position.pkl', 'wb'))
pk.dump(author_position, open('author_position.pkl', 'wb'))

In [52]:
# 上面处理好了数据集 和 作者的坐标
authorBreastSet = set(author_emd.keys())
dataBreastSet = set(dataset_position.keys())

In [53]:
# 转换为dataframe
df_dataset = pd.DataFrame(dataset_position).T
df_author = pd.DataFrame(author_position).T
df_author = df_author.rename({0: 'X', 1: 'Y'}, axis=1)
df_dataset = df_dataset.rename({0: 'X', 1: 'Y'}, axis=1)
df_dataset.to_csv('dataset.csv', encoding='utf-8')
df_author.to_csv('author.csv', encoding='utf-8')

In [72]:
dataset = pd.read_csv('dataset.csv', index_col=0)
author = pd.read_csv('author.csv', index_col=0)

In [108]:
# add author name!
f = open("../data/Authors.csv", "r")
author_name = {}
author_CareerAge = {}
author_PaperNum = {}
author_CitedNum = {}
author_mainAffiliation = {}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] in authorBreastSet:
        author_name[int(line[0])] = line[4]
        author_CareerAge[int(line[0])] = line[1]
        author_PaperNum[int(line[0])] = line[2]
        author_CitedNum[int(line[0])] = line[3]
        author_mainAffiliation[int(line[0])] = str(line[5:]).strip("\"[\]'").replace('\'','').replace('\"','').replace('  ',' ')
f.close()
print(len(author_name))
author['Name'] = pd.Series(author_name)
author['CareerAge'] = pd.Series(author_CareerAge)
author['PaperNum'] = pd.Series(author_PaperNum)
author['CitedNum'] = pd.Series(author_CitedNum)
author['MainAffi'] = pd.Series(author_mainAffiliation)

8699085it [00:08, 1060979.11it/s]

32383


In [57]:
# add dataset name!
f = open("../data/datasetIDName.txt", "r")
print('111')
datasetIDName = {}
for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] in dataBreastSet:
        datasetIDName[int(line[0])] = line[1]
f.close()
print(len(datasetIDName))
dataset['Name'] = pd.Series(datasetIDName)

111


242it [00:00, ?it/s]

227


In [117]:
dataset['clusterID'] = pd.Series(dataset_cluster)
author['clusterID'] = pd.Series(author_cluster)

In [96]:
# add author B2AI!
f = open("../data/B2AiAuthors.csv", "r")
print(1)
n = 5
author_B2AI = {}
for i in authorBreastSet:
    author_B2AI[int(i)] = 0
for line in tqdm(f):
    line = line.strip().split(',')
    if line[0] in authorBreastSet and line[1] != '':
        author_B2AI[int(line[0])] = 1
f.close()
author['isB2AI'] = pd.Series(author_B2AI)
print(sum(author_B2AI.values()))

1


470584it [00:00, 903671.17it/s]

6


In [115]:
# author.loc[author['isB2AI'] == 1]
# df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]
# author
author.loc[(author['clusterID'] == 0) & (author['isB2AI'] == 0)]


,X,Y,Name,clusterID,isB2AI,CareerAge,PaperNum,CitedNum,MainAffi
1197693,-13.302924,2.607302,Vladimir Semiglazov,0,0,53,250,13756,"Petrov Research Institute of Oncology, St. Pet..."
13514515,31.514328,48.308804,Arthur C Louie,0,0,44,4,74,"Cetus Corporation, Emeryville, California 9460..."
13497513,26.720783,50.307144,Daniel D Von Hoff,0,0,48,54,3015,"Department of Medicine, University of Texas He..."
6014112,31.534649,48.332560,Franco Muggia,0,0,58,60,1528,"Norris Cancer Center, University of Southern C..."
1976748,12.341017,42.823277,P E Gormley,0,0,4,1,193,
...,...,...,...,...,...,...,...,...,...
12521583,19.252378,18.229326,Massimo Corsaro,0,0,29,3,10,"Pfizer Oncology, Milan, Italy. massimo.corsaro..."
16334316,19.132893,17.772072,Graciela Rodrigálvarez,0,0,1,1,0,"GEICAM Spanish Breast Cancer Group, Madrid, Sp..."
12865669,19.276052,4.979221,Kai Hong,0,0,4,1,0,"Department of Gynecology, Obstetrics and Gynec..."
2096016,19.278803,4.403614,Xia Sheng,0,0,5,2,13,"Department of Thyroid and Breast Surgery, Ning..."


In [118]:
dataset.to_csv('dataset.csv', encoding='utf-8')
author.to_csv('author.csv', encoding='utf-8')

In [24]:
len(author)


32383